In [2]:
import numpy as np
import pandas as pd
from skimage.measure import regionprops_table
import os
from os import listdir
from os.path import isfile, join

In [2]:
nucleiPath = "segmentations_tilesNuc"

In [3]:
folders = [x[0] for x in os.walk(nucleiPath)]
folders = sorted(folders)

In [4]:
df = pd.DataFrame()
for folder in folders:
    os.chdir(folder)
    files = [f for f in listdir(folder) if isfile(join(folder, f)) if ".DS_Store" not in f if ".npy" in f if "._" not in f]
    for file in files:
        aorta = folder.split("/")[-1]
        age = folder.split("/")[-2]
        nucleiMasks =  np.load(join(folder, file), allow_pickle=True).item()['masks']
        prop_dict = regionprops_table(nucleiMasks, properties = ('label', 'area', 'major_axis_length', 'minor_axis_length'))
        dfHelp = pd.DataFrame(prop_dict)
        dfHelp['age'] = age
        dfHelp['aorta'] = aorta
        dfHelp['file'] = file
        df = pd.concat([df,dfHelp])

In [6]:
df = df.reset_index(drop=True)

In [8]:
df['aspectRatio'] = df['major_axis_length']/df['minor_axis_length']

In [3]:
df = pd.read_csv('results/cellShape/statsNuclei.csv')

In [4]:
dfCount = df.groupby(['age', 'aorta']).count()

In [12]:
dfCount.to_csv('results/cellShape/statsNuclei_Count.csv')

In [6]:
dfMean = df.groupby(['age', 'aorta']).mean()

In [7]:
dfMedian = df.groupby(['age', 'aorta']).median()

In [14]:
df.to_csv("/results/cellShape/statsNuclei.csv", index=False)